In [156]:
#import

import pandas as pd
from sklearn.model_selection import train_test_split
import dataframe_image as dfi
from PIL import Image

In [157]:
#creation of dataset and cleaning of 'NaN' values

path_file = 'transcriptions_with_sex.csv'
data = pd.read_csv(path_file)
data = data.dropna()

In [158]:
#calculating number of individuals, men and women in the dataset

nb_of_individuals_bf_cleaning= len(data)
types_of_sex_bf_cleaning = data['sex'].unique()
data = data.dropna()
types_of_sex = data['sex'].unique()
nb_of_individuals = len(data)
nb_of_men = len(data[data['sex'] == 'homme'])
nb_of_women = len(data[data['sex'] == 'femme'])
nb_of_others = len(data[data['sex'] == 'ambigu'])

print('number of individuals before cleaning : ',nb_of_individuals_bf_cleaning,'types of sex before cleaning : ', types_of_sex_bf_cleaning, sep = '\n')
print('number of individuals : ' ,nb_of_individuals, 'types of sex : ' ,types_of_sex, sep = '\n')
print('number of men : ' , nb_of_men, 'number of ambiguous : ' , nb_of_others, 'number of women : ',nb_of_women, sep = '\n')

number of individuals before cleaning : 
209
types of sex before cleaning : 
['homme' 'femme' 'ambigu']
number of individuals : 
209
types of sex : 
['homme' 'femme' 'ambigu']
number of men : 
114
number of ambiguous : 
7
number of women : 
88


In [159]:
#function to keep only the type of information 

def type_count(x):
    L = []
    for i in x: 
        if len(i) != 0:
            if i[-1] == ':':
                L.append(i)
    return L

In [160]:
#code to determine the different types of information in the 'groundtruth' lines

data['gt'] = data['groundtruth'].apply(lambda x : x.split(' '))
data['gt'] = data['gt'].apply(type_count)
data['gt'] = data['gt'].apply(lambda x : ''.join(x))

In [161]:
#code to determine the different types of information in the 'prediction' lines

data['gt'] = data['prediction'].apply(lambda x : x.split(' '))
data['gt'] = data['gt'].apply(type_count)
data['gt'] = data['gt'].apply(lambda x : ' '.join(x))
print(data['gt'].unique())
print(len(data['gt'].unique()))

['nom: prénom: date_naissance: lieux_naissance: employeur:'
 'nom: prénom: date_naissance: relation:'
 'nom: prénom: date_naissance: lieux_naissance: profession:'
 'nom: prénom: date_naissance: lieux_naissance: relation:'
 'nom: prénom: date_naissance: lieux_naissance:'
 'nom: prénom: date_naissance: lieux_naissance: relation: profession:'
 'nom: prénom: date_naissance: lieux_naissance: relation: employeur:'
 'nom: prénom: date_naissance:'
 'nom: prénom: date_naissance: lieux_naissance: état_civil: relation:'
 'nom: prénom: date_naissance: lieux_naissance: état_civil: profession: employeur:'
 'nom: prénom: date_naissance: lieux_naissance: état_civil: relation: profession: employeur:'
 'prénom: date_naissance: lieux_naissance: relation:'
 'nom: prénom: date_naissance: lieux_naissance: relation: employeur: prénom: date_naissance: lieux_naissance:'
 'nom: prénom: date_naissance: état_civil: employeur:'
 'nom: prénom: date_naissance: lieux_naissance: relation: profession: employeur:'
 'nom

In [162]:
#functions to filter firstname out of datasets

def firstname_keep(x):
    y = 0
    if x[0][0] == 's':
        y = x[3]
    if x[0][0] == 'f':
        y = x[1]
    return y

def firstname_keep_p(x):
    y = 0
    if x[0][0] == 'n':
        y = x[3]
    if x[0][0] == 'p':
        y = x[1]
    return y

In [163]:
#code to create tables of occurences of firstname in 'prediction' and 'groundtruth' categories

data['firstname'] = data['groundtruth'].apply(lambda x : x.split(' '))
data['firstname'] = data['firstname'].apply(firstname_keep)
list_firstname_gt = data['firstname'].unique()

occurences_by_firstname = []
for i in list_firstname_gt:
    occurences_by_firstname.append(len(data[data['firstname'] == i]))

table_firstname = pd.DataFrame({'Firstnames' : list_firstname_gt, 'Occurences' : occurences_by_firstname})

data['firstname_p'] = data['prediction'].apply(lambda x : x.split(' '))
data['firstname_p'] = data['firstname_p'].apply(firstname_keep_p)
list_firstname_p = data['firstname_p'].unique()

occurences_by_firstname_p = []
for i in list_firstname_p:
    occurences_by_firstname_p.append(len(data[data['firstname_p'] == i]))

table_firstname_p = pd.DataFrame({'Firstnames' : list_firstname_p, 'Occurences' : occurences_by_firstname_p})

table_firstname_p = table_firstname_p.sort_values(by='Occurences', ascending=False)
table_firstname = table_firstname.sort_values(by='Occurences', ascending=False)

table_firstname_p = table_firstname_p[:20]
table_firstname_p = table_firstname_p.reset_index(drop=True)
table_firstname = table_firstname[:20]
table_firstname = table_firstname.reset_index(drop=True)

In [164]:
#code creating tables containing occurences of firstname male and female in the french population

path_file_2 = 'firstname_with_sex.csv'
data_2 = pd.read_csv(path_file_2,sep = ';')
data_h = data_2.drop('female', axis=1)
data_f = data_2.drop('male', axis=1)
data_h = data_h.sort_values(by='male', ascending=False)
data_f = data_f.sort_values(by='female', ascending=False)
data_h = data_h[:20]
data_h = data_h.reset_index(drop=True)
data_f = data_f[:20]
data_f = data_f.reset_index(drop=True)

In [165]:
#code to create images and concatenating them 

dfi.export(data_h,'data_h.png')
dfi.export(data_f,'data_f.png')
dfi.export(table_firstname, 'table_firstname_1.png')
dfi.export(table_firstname_p, 'table_firstname_p_1.png')
image1 = Image.open('data_h.png')
image4 = Image.open('data_f.png')
image2 = Image.open('table_firstname_1.png')
image3 = Image.open('table_firstname_p_1.png')
total_width = image1.width + image4.width + image2.width + image3.width 
max_height = max(image1.height, image4.height, image2.height, image3.height)
new_image = Image.new('RGB', (total_width, max_height))
x_offset = 0
new_image.paste(image1, (x_offset, 0))
x_offset += image1.width
new_image.paste(image4, (x_offset, 0))
x_offset += image4.width
new_image.paste(image2, (x_offset, 0))
x_offset += image2.width
new_image.paste(image3, (x_offset, 0))
new_image.save('merged_horizontal.png')